In [1]:
import numpy as np
from numpy.typing import NDArray

In [74]:
class Algorithm:
    def __init__(self, r):
        self.r = r
    
    def initialize(self):
        
        self.U = np.concatenate([self.e(i, self.n).reshape(-1, 1) for i in range(self.r)], axis=-1).T # shape(n, r)

        self.P = np.eye(self.r) # shape (r,r)
        self.Q = np.eye(self.r) # shape (r, r)
        self.lambdas = np.ones(self.r) # shape (r)
        self.W = X.T @ np.concatenate([self.e(i, self.n).reshape(-1, 1) for i in range(self.r)], axis=-1) # shape (N, r)
        self.t = np.array([self.X[:, p].T @ self.U.T @ self.P @ self.U @ self.X[:, p] for p in range(self.N)])
        assert (np.all([np.isclose(self.t[p], np.sum(self.X[:, p][:r]**2)) for p in range(self.N)]))
    
    def check_shapes(self):
        assert self.X.shape == (n, N)
        assert self.y.shape == (N,)
        assert self.U.shape == (r, n)
        assert self.P.shape == (r, r)
        assert self.Q.shape == (r, r)
        assert self.lambdas.shape == (r,)
        assert self.W.shape == (N, r)
        assert self.t.shape == (N,)
    
    def fit(self, X, y):
        assert X.shape[1] == y.shape[0]
        assert r <= n
        
        self.n = X.shape[0]
        self.N = X.shape[1]
        self.X = X
        self.y = y
        
        self.initialize()
        self.run()
        
    def v_hat(self, k):
        return (2*self.lambdas[k]/self.N)*(self.X @ ((self.y-self.t)*self.W[:, k]) - self.U.T @ self.Q @ (self.W.T @ ((self.y-self.t)*self.W[:, k])))
   
    def v(self, k):
        return self.v_hat(k)/np.linalg.norm(self.v_hat(k), ord=2)
    
    def U_plus(self, k, gamma):
        print(self.U.shape)
        print(self.Q[:, k].shape)
        return self.U + (((np.cos(gamma)-1)*self.U.T @ self.Q[:, k] + np.sin(gamma)*self.v(k)).reshape(-1, 1) @ self.Q[:, k].T.reshape(1, -1)).T
    
    def h(self, k, gamma):
        return (1/(2*self.N))*np.linalg.norm(self.y-self.t-2*self.lambdas[k]*self.s(k, gamma) - self.lambdas[k]*(self.s(k, gamma)*self.s(k, gamma)), ord=2)
    
    def find_optimal_gamma(self, k):
        num_points = 1000
        x_values = np.linspace(0, 2*np.pi, num_points)
        y_values = [self.h(k, x) for x in x_values]

        # Find the x value that minimizes the objective function
        optimal_x = x_values[np.argmin(y_values)]
        optimal_y = np.min(y_values)

        return optimal_x
    
    def z(self, k):
        return X.T @ self.v(k)
    
    @staticmethod
    def e(k, r):
        if k > r:
            raise ValueError("k should be less than or equal to r")

        # Create a zero vector of size r
        e_kr = np.zeros(r)

        # Set the k-th entry to 1
        e_kr[k] = 1

        return e_kr
    
    def s(self, k, gamma):
        return (np.cos(gamma)-1)*self.W[:, k] + np.sin(gamma)*self.z(k)
    
    def run(self, num_iterations=100):
        
        for t in range(num_iterations):
            for l in range(self.n):
                for k in range(self.r):
                    
                    optimal_gamma = self.find_optimal_gamma(k)

                    self.U = self.U_plus(k, optimal_gamma)
                    self.P = self.P
                    

                    s = self.s(k, optimal_gamma)
                    
                    self.W[:, k] = self.X.T @ (s + self.U.T @ self.Q[:, k]) # X.T shape = N, n
                    self.t = self.t + 2*self.lambdas[k]*s + self.lambdas[k]*(s*s)
        #             UQ[:, k] += s

In [75]:
# def h_k(gamma, N, y, t, lambdas, W, ):
# #     return (1/(2*N))*np.linalg.norm(y, ord=2)

In [76]:
n, N, r = 7, 10, 4

In [77]:
# Inputs
X = np.random.random((n, N)) # shape (n, N)
y = np.zeros(N) # shape (N)

In [78]:
algo = Algorithm(r)
algo.fit(X, y)

(4, 7)
(4,)


ValueError: operands could not be broadcast together with shapes (10,) (7,) 

In [143]:
# # Inicialization

# U = np.concatenate([e(i, n).reshape(-1, 1) for i in range(r)], axis=-1).T # shape(n, r)

# P = np.eye(r) # shape (r,r)
# Q = np.eye(r) # shape (r, r)
# lambdas = np.ones(r) # shape (r)
# W = X.T @ np.concatenate([e(i, n).reshape(-1, 1) for i in range(r)], axis=-1) # shape (N, r)
# t = np.array([X[:, p].T @ U.T @ P @ U @ X[:, p] for p in range(N)])

In [147]:
# def v_hat(k):
#     return (2*lambdas[k]/N)*(X @ ((y-t)*W[:, k]) - UQ @ (W.T @ ((y-t)*W[:, k])))

In [148]:
# def v(k):
#     v_hat = v_hat(k)
    
#     return v_hat/np.linalg.norm(v_hat)**2

In [149]:
# def U_plus(gamma, v, k):
#     return U.T + ((np.cos(gamma)-1)*U.T @ Q[:, k] + np.sin(gamma)*v).reshape(-1, 1) @ Q[:, k].reshape(1, -1)

In [150]:
# def h(gamma, k):
    
#     v_hat = (2*lambdas[k]/N)*(X @ ((y-t)*W[:, k]) - U.T @ Q @ (W.T @ ((y-t)*W[:, k])))
            
#     v = v_hat/np.linalg.norm(v_hat)**2
#     z = X.T @ v
#     s = (np.cos(gamma)-1)*W[:, k] + np.sin(gamma)*z
#     return (1/(2*N)) * np.linalg.norm(y-t-2*lambdas[k]*s - lambdas[k]*(s*s), ord=2)**2

In [151]:
# def find_optimal_gamma(k):
#     num_points = 1000
#     x_values = np.linspace(0, 2*np.pi, num_points)
#     y_values = [h(x, k) for x in x_values]

#     # Find the x value that minimizes the objective function
#     optimal_x = x_values[np.argmin(y_values)]
#     optimal_y = np.min(y_values)
    
#     return optimal_x

In [152]:
# for t in range(num_iterations):
#     for l in range(n):
#         for k in range(r):
            
#             v_hat = (2*lambdas[k]/N)*(X @ ((y-t)*W[:, k]) - U.T @ Q @ (W.T @ ((y-t)*W[:, k])))
            
#             assert v_hat.shape == (n,)
            
#             v = v_hat/np.linalg.norm(v_hat)**2
            
#             z = X.T @ v
            
#             assert z.shape == (N,)
            
#             optimal_gamma = find_optimal_gamma(k)
            
#             U = U_plus(optimal_gamma, v, k)
#             P = P
            
            
            
#             s = (np.cos(optimal_gamma)-1)*W[:, k] + np.sin(optimal_gamma)*z
            
#             W[:, k] = X.T @ (s + U.T @ Q[:, k]) # X.T shape = N, n
#             t = t + 2*lambdas[k]*s + lambdas[k]*(s*s)
# #             UQ[:, k] += s

(10,)


/var/folders/7x/w344fx1j18q71m51_grbtgjc0000gn/T/ipykernel_61837/1935296653.py:9: RuntimeWarning: invalid value encountered in divide
  v = v_hat/np.linalg.norm(v_hat)**2
/var/folders/7x/w344fx1j18q71m51_grbtgjc0000gn/T/ipykernel_61837/1223925686.py:5: RuntimeWarning: invalid value encountered in divide
  v = v_hat/np.linalg.norm(v_hat)**2


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 7)